In [1]:
!nvidia-smi


Thu Dec 21 19:17:57 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.12                 Driver Version: 546.12       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   37C    P3              12W /  35W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:

from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

c:\Users\omkar\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Nov__3_17:51:05_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.3, V12.3.103
Build cuda_12.3.r12.3/compiler.33492891_0


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [6]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model_pegases = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)#.to(device)

In [7]:
%pwd

'c:\\Users\\omkar\\Desktop\\Omkar\\Projects\\NLP\\Text-Summarizer\\research'

In [8]:
import os
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\omkar\\Desktop\\Omkar\\Projects\\NLP\\Text-Summarizer'

In [10]:
os.chdir("C:\\Users\\omkar\\Desktop\\Omkar\\Projects\\NLP\\Text-Summarizer\\artifacts\\data_ingestion")

In [11]:
%pwd

'C:\\Users\\omkar\\Desktop\\Omkar\\Projects\\NLP\\Text-Summarizer\\artifacts\\data_ingestion'

In [12]:
dataset = load_from_disk('samsum_dataset')

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [14]:
def convert_examples_to_features(example_batch):
    prefix = "summarize: "
    inputs = [prefix + doc for doc in example_batch["dialogue"]]
    input_encodings = tokenizer(inputs , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    

In [15]:
dataset_pt = dataset.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

c:\Users\omkar\anaconda3\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 3468.29 examples/s]


In [16]:
dataset_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [17]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegases)
     

In [7]:
import textSumarizer

ModuleNotFoundError: No module named 'textSumarizer'

In [40]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='T5-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16,fp16=True,learning_rate=2e-5)



In [41]:

trainer = Trainer(model=model_pegases, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["train"], 
                  eval_dataset=dataset_pt["validation"])

In [43]:
trainer.train()

  1%|▏         | 12/920 [01:26<1:48:30,  7.17s/it]

                                             

{'loss': 2.7376, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.01}


{'loss': 2.7706, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.02}


{'loss': 2.7684, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.03}


{'loss': 2.7193, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.04}


...

{'loss': 2.0724, 'learning_rate': 1.0476190476190478e-06, 'epoch': 0.98}


{'loss': 1.9893, 'learning_rate': 5.714285714285715e-07, 'epoch': 0.99}



                                              
100%|██████████| 920/920 [29:30<00:00,  1.92s/it]

{'loss': 2.0662, 'learning_rate': 9.523809523809525e-08, 'epoch': 1.0}
{'train_runtime': 1770.9349, 'train_samples_per_second': 8.319, 'train_steps_per_second': 0.519, 'train_loss': 2.260293888009113, 'epoch': 1.0}


TrainOutput(global_step=920, training_loss=2.260293888009113, metrics={'train_runtime': 1770.9349, 'train_samples_per_second': 8.319, 'train_steps_per_second': 0.519, 'train_loss': 2.260293888009113, 'epoch': 1.0})

In [44]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [48]:

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

ConnectionError: Couldn't reach https://raw.githubusercontent.com/huggingface/datasets/2.14.6/metrics/rouge/rouge.py (ConnectionError(MaxRetryError("HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /huggingface/datasets/2.14.6/metrics/rouge/rouge.py (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001A13FAFE0D0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))")))

In [ ]:

score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f't5'] )

In [ ]:
model_pegases.save_pretrained("T5-samsum-model")


In [ ]:
tokenizer.save_pretrained("tokenizer")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")


In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["dialogue"]

reference = dataset["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

## 
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])